In [29]:
from helpers import npNicePrint
import numpy as np
import ast

#helpers.npNicePrint(arr)


data = """broadcaster -> a
%a -> inv, con
&inv -> b
%b -> con
&con -> output"""

with open("input_20.txt", 'r') as f:
    data = f.read().strip()

modules = {}

for row in data.split("\n"):
    mdl, targets= row.split(" -> ")
    targets = targets.split(", ")
    mdl_type = None
    if "%" in mdl:
        mdl_type = "%"
        mdl = mdl[1:]
    elif "&" in mdl:
        mdl_type = "&"
        mdl = mdl[1:]
        
    
    if not mdl in modules:
        if mdl_type == "&":
            modules[mdl] = {"value": 0, "mdl_type": mdl_type, "targets": targets, "inputs": {}}
        else:
            modules[mdl] = {"value": 0, "mdl_type": mdl_type, "targets": targets}

for row in data.split("\n"):
    mdl, targets= row.split(" -> ")
    targets = targets.split(", ")
    
    if "%" in mdl:
        mdl_type = "%"
        mdl = mdl[1:]
    elif "&" in mdl:
        mdl_type = "&"
        mdl = mdl[1:]

    for target in targets:
        if not target in modules:
            modules[target] = {"value": 0, "mdl_type": None}
        if 'inputs' in modules[target].keys():
            if not mdl in modules[target]['inputs']:
                modules[target]['inputs'][mdl] = 0

    
            
modules

{'mh': {'value': 0, 'mdl_type': '%', 'targets': ['rz']},
 'nd': {'value': 0, 'mdl_type': '%', 'targets': ['jx']},
 'xt': {'value': 0, 'mdl_type': '%', 'targets': ['cx']},
 'dp': {'value': 0, 'mdl_type': '%', 'targets': ['mh']},
 'pz': {'value': 0, 'mdl_type': '%', 'targets': ['zg', 'bf']},
 'rp': {'value': 0, 'mdl_type': '%', 'targets': ['jb', 'bf']},
 'jb': {'value': 0, 'mdl_type': '%', 'targets': ['bf', 'kp']},
 'rj': {'value': 0, 'mdl_type': '%', 'targets': ['xt', 'cx']},
 'hg': {'value': 0, 'mdl_type': '%', 'targets': ['dl', 'bf']},
 'pt': {'value': 0, 'mdl_type': '%', 'targets': ['gm', 'vv']},
 'pf': {'value': 0, 'mdl_type': '%', 'targets': ['xk', 'qr']},
 'cv': {'value': 0, 'mdl_type': '%', 'targets': ['jp', 'cx']},
 'zg': {'value': 0, 'mdl_type': '%', 'targets': ['bb']},
 'qn': {'value': 0, 'mdl_type': '%', 'targets': ['gm', 'bh']},
 'kp': {'value': 0, 'mdl_type': '%', 'targets': ['pz']},
 'kg': {'value': 0, 'mdl_type': '%', 'targets': ['gm', 'pt']},
 'sl': {'value': 0, 'mdl_typ

In [30]:
from collections import deque

nicing = {0: "-low->", 1: "-high->"}



low_pulses = 0
high_pulses = 0

for x in range(1000):
    signal_queue = deque()
    signal_queue.append(["button", "broadcaster", 0])
    
    while len(signal_queue) > 0:
        previous_node, next_node, value = signal_queue.popleft()
        #print(previous_node, nicing[value], next_node)
        if value == 0:
            low_pulses += 1
        elif value == 1:
            high_pulses += 1
        if next_node == "broadcaster":
            modules[next_node]['value'] = value
            for target in modules[next_node]['targets']:
                signal_queue.append([next_node, target, modules[next_node]['value']])
            continue
        mdl_type = modules[next_node]['mdl_type']
        if mdl_type == "%" and value == 0:
            modules[next_node]['value'] = (modules[next_node]['value'] + 1) % 2
            for target in modules[next_node]['targets']:
                signal_queue.append([next_node, target, modules[next_node]['value']])

        if mdl_type == "&":
            modules[next_node]['inputs'][previous_node] = value
            for target in modules[next_node]['targets']:
                signal_queue.append([next_node, target, int(not all(value == 1 for value in modules[next_node]['inputs'].values()))])

print(low_pulses, high_pulses, low_pulses * high_pulses)           

17575 42103 739960225
